# Twitter Sentiment Analysis 

# ARTG Project - By Team Maple

**<font color="blue">Import libraries</font>**  

**<font color="red">Code:</font>**

In [ ]:
import csv
import sys
!conda install --yes --prefix {sys.prefix} -c conda-forge tweepy
import tweepy

**<font color="blue">Step 1: Twitter crawl & downloading data</font>**  

**<font color="red">Code:</font>**

In [ ]:
consumer_key = 'kmflAhDlFLYvVLzUhUTv5Agbw'
consumer_secret = 'D8Giq3GMjIMaBQN0Qw63tsoHpajA7hNKy2ouo1XeqQP46SP38C'
access_token = '4266439228-U0ySetwuTNEtz3ZGdyPWKVrfIGKY866EMIrzNbN'
access_token_secret = 'CVynCeCX22NtK8iygTHGWjota2JhGTfPgk2CO1MUnPfBK'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, timeout=1000)

In [ ]:
# Open/create a file to append data to
csvFile = open('testgf.csv', 'a')

In [ ]:
# Used csv writer
csvWriter = csv.writer(csvFile)

In [ ]:
for tweet in tweepy.Cursor(api.search,
                           q="#blacklivesmatter",
                           # Data only available for last 7 days. I mined tweets for 4 days - 24,25,26,27 June
                           since="2020-06-24",
                           # Until 28 means  <28 i.e. <=27
                           until="2020-06-28",
                           lang="en",
                           count=100).items():
    # Write a row to the CSV file. We used encode UTF-8
    csvWriter.writerow([tweet.created_at, tweet.text.encode('utf-8')])
    print(tweet.created_at, tweet.text)
csvFile.close()

**<font color="blue">Step 2: Cleaning the tweets</font>**  

**<font color="red">Code:</font>**